In [1]:
from tensorflow.keras.models import load_model
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw

from itertools import product

import sys
from core.UNet import UNet
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

2.5.0-rc3


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis.Configuration()

self.trained_model_path: D:\sample250\U-Net_13\saved_models\UNet\lakes_20230818-2031_AdaDelta_dice_loss_b5_012345_512.h5


In [4]:
# Load a pretrained model 
OPTIMIZER = adaDelta
# OPTIMIZER = adam
OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

Instructions for updating:
Use tf.keras.mixed_precision. There is a guide at https://www.tensorflow.org/guide/mixed_precision. Alternatively, `tf.compat.v1.mixed_precision.enable_mixed_precision_graph_rewrite` can be used, but this is not recommended for TF2 code.
  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)


In [10]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,padding):
    currValue = res[row:row+he, col:col+wi]
    newPredictions = prediction[:he, :wi]
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row:row+he, col:col+wi] =  np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row:row+he, col:col+wi] = np.maximum(currValue, newPredictions)
    elif operator == 'padding':
        newPredictions = newPredictions[padding:he-padding, padding:wi-padding]
        res[row+padding:row+he-padding, col+padding:col+wi-padding] = newPredictions
    else:
        res[row:row+he, col:col+wi] = newPredictions  
    return (res)

In [11]:
# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,padding):
    tm = np.stack(batch, axis = 0)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,PADDING)
        mask = addTOResult(mask, p, row, col, he, wi,operator,padding)  
    return mask
    
def detect_lake(fullPath, NDWI_img, bands_img, operator = 'max',width=512, height=512, stride = 256,padding=100):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, 5),-1.0)#Add -1 padding in case of corner images
    
        read_ndwi_img=NDWI_img.read(window=window)/100
        read_bands_img = bands_img.read(window=window)/1000
        
        temp_im = np.concatenate((read_ndwi_img,read_bands_img))
        temp_im =  np.transpose(temp_im, axes=(1,2,0))
        
        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,padding)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,padding)
        batch = []
        batch_pos = []

    return(mask, meta)

In [12]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
        
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')

def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [13]:
# Predict trees in the all the files in the input image dir 
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = []
for root, dirs, files in os.walk(config.input_image_dir):
    for file in files:
        if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
             all_files.append((os.path.join(root, file), file))
print(all_files)

operator='padding'
for fullPath, filename in all_files:
    outputFile = os.path.join(config.output_dir, filename.replace(config.ndwi_fn_st, config.output_prefix + operator) )
    if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
        with rasterio.open(fullPath) as NDWI:
            with rasterio.open(fullPath.replace(config.ndwi_fn_st,config.bands_fn_st)) as bands:
                print(fullPath)
                detectedMask, detectedMeta = detect_lake(fullPath,NDWI,bands,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE)
                writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
                vecter_path =outputFile.replace('tif','shp')
                field_name = "class"
#            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
                ignore_values = [0]
                raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
    else:
        print('File already analysed!', fullPath)
        
print('finish')

6it [00:00, 57.67it/s]

[('D:\\sample250\\test_img\\ndwi_int8_65.tif', 'ndwi_int8_65.tif'), ('D:\\sample250\\test_img\\ndwi_int8_43.tif', 'ndwi_int8_43.tif'), ('D:\\sample250\\test_img\\ndwi_int8_56.tif', 'ndwi_int8_56.tif')]
D:\sample250\test_img\ndwi_int8_65.tif
the size of current NDWI_img 11355 11356


1936it [04:50,  6.67it/s]


Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!


6it [00:00, 59.39it/s]

...

Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!


6it [00:00, 55.25it/s]

D:\sample250\test_img\ndwi_int8_56.tif
the size of current NDWI_img 11355 11356


1936it [04:47,  6.75it/s]


Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
finish
